# MATH 441 Optimization Problems

**February 28, 2024**

In [30]:
import numpy as np
import pulp

## Sudoku as a 0-1 Integer Programming Problem

Let $x_{ijk} = 1$ if entry at row $i$ and column $j$ is filled with digit $k$ and $0$ if not. The indices are $i=0,\dots,8$, $j=0,\dots,8$ and $k=1,\dots,9$.

Each entry is assigned exactly one digit:

$$
\sum_{k=1}^9 x_{ijk} = 1 \ , \ \ \text{for } i=0,\dots,8 \ , \ \ j=0,\dots,8
$$

Each row contains each digit exactly once:

$$
\sum_{j=0}^8 x_{ijk} = 1 \ , \ \ \text{for } i=0,\dots,8 \ , \ \ k=1,\dots,9
$$

Each column contains each digit exactly once:

$$
\sum_{i=0}^8 x_{ijk} = 1 \ , \ \ \text{for } j=0,\dots,8 \ , \ \ k=1,\dots,9
$$

Each submatrix contains each digit exactly once:

$$
\sum_{i'=0}^{2} \sum_{j'=0}^{2} x_{3m+i',3n+j',k} = 1 \ , \ \ \text{for } m=0,1,2 \ , \ \ n=0,1,2 \ , \ \ k=0,\dots,8
$$

## Algorithms for Exact Solutions

We asked ChatGPT to write Python code to find solutions of Sudoku puzzles. It gave us something that used recursion. We tried to write our own code that did not use recursion but we couldn't get it to work. Recursion (or something equivalent) is essential to the backtracking algorithm used to solve sudoku puzzles.

## PuLP Solution

Let's use PuLP to solve the integer programming formulation above.

In [29]:
S = [[3,0,0,8,0,0,0,0,1],
     [0,0,0,0,0,2,0,0,0],
     [0,4,1,5,0,0,8,3,0],
     [0,2,0,0,0,1,0,0,0],
     [8,5,0,4,0,3,0,1,7],
     [0,0,0,7,0,0,0,2,0],
     [0,8,5,0,0,9,7,4,0],
     [0,0,0,1,0,0,0,0,0],
     [9,0,0,0,0,7,0,0,6]]

prob = pulp.LpProblem()
rows = range(0,9)
cols = range(0,9)
vals = range(1,10)
X = pulp.LpVariable.dicts("X",(rows,cols,vals),cat='Binary')

# Each entry is assigned exactly one value
for i in rows:
    for j in cols:
        prob += sum([X[i][j][k] for k in vals]) == 1
        
# Each row contains each digit exactly once
for i in rows:
    for k in vals:
        prob += sum([X[i][j][k] for j in cols]) == 1

# Each column contains each digit exactly once
for j in cols:
    for k in vals:
        prob += sum([X[i][j][k] for i in rows]) == 1

# Each submatrix contains each digit exactly once
for r in [0,1,2]:
    for c in [0,1,2]:
        for k in vals:
            prob += sum([X[3*r + i][3*c + j][k] for j in [0,1,2] for i in [0,1,2]]) == 1

# Enter values given in the puzzle
for i in rows:
    for j in cols:
        if (S[i][j] != 0):
            k = S[i][j]
            prob += X[i][j][k] == 1

prob.solve(pulp.PULP_CBC_CMD(msg=0));

# Extract values from the variable X[i][j][k] and print the solved puzzle
solution = np.zeros((9,9))
for i in rows:
    for j in cols:
        for k in vals:
            if X[i][j][k].value() == 1:
                solution[i,j] = k

print(solution)

[[3. 6. 7. 8. 9. 4. 2. 5. 1.]
 [5. 9. 8. 3. 1. 2. 6. 7. 4.]
 [2. 4. 1. 5. 7. 6. 8. 3. 9.]
 [7. 2. 3. 9. 8. 1. 4. 6. 5.]
 [8. 5. 6. 4. 2. 3. 9. 1. 7.]
 [4. 1. 9. 7. 6. 5. 3. 2. 8.]
 [1. 8. 5. 6. 3. 9. 7. 4. 2.]
 [6. 7. 2. 1. 4. 8. 5. 9. 3.]
 [9. 3. 4. 2. 5. 7. 1. 8. 6.]]
